In [1]:
# Not needed if pysleep is pip installed (i.e. you probably dont need these lines)
import sys, os
file_dir = os.path.abspath('')
sys.path.insert(0, file_dir+'/../')

In [2]:
#%% Import the tools we need
from mednickdb_pysleep import sleep_features, pysleep_defaults, pysleep_utils, scorefiles, \
    sleep_architecture, frequency_features, artifact_detection
import pandas as pd
import numpy as np
import yaml
import mne
import warnings
import datetime
warnings.filterwarnings("ignore")

In [3]:
#setup location of files and save locations
edf_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/'
scorefile_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/sleep_scoring/'
edf_filenames = ['Sid1_edf.edf']
scorefile_filenames = ['Sid1_epoch.csv']
ids = ['1']
study_settings_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/study_settings/ExampleStudy2_study_settings.yaml'
save_base_path = 'C:/Users/bdyet/Desktop/ExampleStudy2/'
nighttime_split_method = 'quartiles' #None or quartiles
spindle_algo = 'Wamsley2012'


In [ ]:
#%% loop through edf's, extract band power, spindles and slow osc (per stage)
edf_filepaths = [edf_base_path + edf_filename for edf_filename in edf_filenames]
scoring_filepaths = [scorefile_base_path + scorefile_filename for scorefile_filename in scorefile_filenames]
features_cont = []
band_power_cont = []

study_settings = yaml.safe_load(open(study_settings_path,'r+'))

for edf_filepath, scorefile_filepath, id_ in zip(edf_filepaths, scoring_filepaths, ids):   
    print('Working on',edf_filepath)
    
    #%%Get stage info
    epoch_stages, epochoffset, starttime = scorefiles.extract_epochstages_from_scorefile(scorefile_filepath, 
                                                                                         study_settings['stage_map'])
    epoch_stages = scorefiles.score_wake_as_waso_wbso_wase(epoch_stages)

    #get the start and end of where we want to extract spindles from (lights off->lights on)
    lights_off_secs, lights_on_secs, \
    _, epoch_stages = sleep_architecture.lights_on_off_and_sleep_latency(epoch_stages,
                                                                         epoch_sync_offset_seconds=epochoffset)
    
    edf = mne.io.read_raw_edf(edf_filepath)
    all_eeg_chans = [v for v in study_settings['known_eeg_chans'].keys() if v in edf.ch_names]

    print(datetime.datetime.now(), '\tDetecting artifacts')
    epochs_with_artifacts = artifact_detection.detect_artifacts(edf_filepath=filepath,
                                                                epochstages=epochstages,
                                                                start_offset=start_offset,
                                                                end_offset=end_offset,
                                                                chans_to_consider=all_eeg_chans)
    print('\t\tRecord contains ',100*len(epochs_with_artifacts)/len(epoch_stages)),'% bad epochs that will be ignored'

    print(datetime.datetime.now(), '\tBand power extraction started')
    band_power = frequency_features.extract_band_power(edf_filepath=filepath,
                                                       start_time=start_offset,
                                                       end_time=end_offset,
                                                       chans_to_consider=all_eeg_chans
                                                            if study_settings['chans_for_band_power']=='all'
                                                            else study_settings['chans_for_band_power'],
                                                       epoch_len=pysleep_defaults.band_power_epoch_len)

    band_power_per_epoch = frequency_features.extract_band_power_per_epoch(band_power,
                                                                           epoch_len=pysleep_defaults.epoch_len)


    band_power_w_stage = frequency_features.assign_band_power_stage(band_power_per_epoch,
                                                                    epochstages, bad_epochs=epochs_with_artifacts)

    if len(epochstages) > pyparse_defaults.max_nap_len_in_epochs:
        per_quartile = True
        if nighttime_split_method == 'quartiles':
            band_power_w_stage, _ = pysleep_utils.assign_quartiles(band_power_w_stage, epochstages)
            groupby = ['quartile', 'stage', 'chan', 'band']
        else:
            if nighttime_split_method is not None:
                raise NotImplementedError('Only quartiles (or None) night split method is implemented')
    else:
        per_quartile = False
        groupby = ['stage', 'chan', 'band']

    band_power_w_stage = band_power_w_stage.drop(['onset','duration'], axis=1)
    band_power_w_stage_to_mean = band_power_w_stage.loc[band_power_w_stage['stage'].isin(pysleep_defaults.stages_to_consider), :]
    power_df = band_power_w_stage_to_mean.groupby(groupby).agg(np.nanmean)
    power_df['power'] = pysleep_utils.trunc(power_df['power'], 3)
    power_df.index = ['_'.join(idx) for idx in power_df.index.values]
    band_power_data = power_df.T.iloc[0,:].to_dict()

    mins_df = sleep_architecture.sleep_stage_architecture(epochstages,
                                                          epochs_to_ignore=epochs_with_artifacts,
                                                          return_type='dataframe',
                                                          per_quartile=per_quartile)


    #%% Sleep Features
    features_detected = []



    if 'chans_for_spindles' in study_settings and study_settings['chans_for_spindles']:
        chans_to_consider = all_eeg_chans if study_settings['chans_for_spindles'] == 'all' else study_settings['chans_for_spindles']
        data = sleep_features.load_and_slice_data_for_feature_extraction(edf_filepath=filepath,
                                                                         epochstages=epochstages,
                                                                         start_offset=0,
                                                                         end_offset=3000,
                                                                         chans_to_consider=chans_to_consider)
        spindles = sleep_features.detect_spindles(data, start_offset=start_offset, algo=pyparse_defaults.spindle_algo)
        spindles = sleep_features.assign_stage_to_feature_events(spindles, epochstages)
        features_detected.append(spindles)

    if 'chans_for_slow_osc' in study_settings and study_settings['chans_for_slow_osc']:
        chans_to_consider = all_eeg_chans if study_settings['chans_for_slow_osc'] == 'all' else study_settings[
            'chans_for_slow_osc']
        data = sleep_features.load_and_slice_data_for_feature_extraction(edf_filepath=filepath,
                                                                         epochstages=epochstages,
                                                                         start_offset=0,
                                                                         end_offset=3000,
                                                                         chans_to_consider=chans_to_consider)
        sos = sleep_features.detect_slow_oscillation(data, start_offset=start_offset)
        sos = sleep_features.assign_stage_to_feature_events(sos, epochstages)
        features_detected.append(sos)

    if 'chans_for_rems' in study_settings and study_settings['chans_for_rems'] and pysleep_defaults.load_matlab_detectors:
        eog_chans = list(study_settings['known_eog_chans'].values)
        if 'LOC' not in eog_chans or 'ROC' not in eog_chans:
            raise ParseError('Cannot extract REM without LOC and ROC channels')
        data = sleep_features.load_and_slice_data_for_feature_extraction(edf_filepath=filepath,
                                                                         epochstages=epochstages,
                                                                         start_offset=0,
                                                                         end_offset=3000,
                                                                         chans_to_consider=['LOC','ROC'],
                                                                         stages_to_consider=['rem'])
        rems = sleep_features.detect_rems(edf_filepath=filepath, data=data)
        rems = sleep_features.assign_stage_to_feature_events(rems, epochstages)
        features_detected.append(rems)

    features_df = pd.concat(features_detected, axis=0, sort=False)
    if features_df.shape[0] == 0:
        sleep_feature_data = {}
    else:
        if len(epochstages) > pyparse_defaults.max_nap_len_in_epochs:
            features_df, _ = pysleep_utils.assign_quartiles(features_df, epochstages)
            groupby = ['quartile', 'stage', 'chan', 'description']
        else:
            groupby = ['stage', 'chan', 'description']

        features_per_stage = sleep_features.sleep_feature_variables_per_stage(features_df,
                                                                              mins_in_stage_df=mins_df,
                                                                              av_across_channels=False,
                                                                              stages_to_consider=pysleep_defaults.stages_to_consider)

        features_per_stage = features_per_stage.apply(lambda x: pd.to_numeric(x, errors='ignore'))
        features_per_stage = features_per_stage.groupby(groupby).agg(np.nanmean)
        features_per_stage = features_per_stage.apply(lambda x: pysleep_utils.trunc(x,3))
        features_per_stage['id'] = id_
        features_per_stage = features_per_stage.reset_index().set_index('id').reset_index()
    
    features_cont.append(features_per_stage)
    print(datetime.datetime.now(), '\tSleep Features extraction finished')
    

Working on C:/Users/bdyet/Desktop/ExampleStudy2/raw_sleep_eeg/Sid1_edf.edf
2019-06-23 14:49:46.634577 	Band power extraction started
2019-06-23 14:50:50.529854 	Band power extraction finished
2019-06-23 14:50:50.529854 	Sleep Features extraction starting


In [24]:
#Combine all spindles into a single dataframe and save to csv
all_band_info = pd.concat(band_power_cont, axis=0)
all_band_info.to_csv(save_base_path + 'band_power_data.csv', index=False)

all_feature_info = pd.concat(features_cont, axis=0)
all_feature_info.to_csv(save_base_path + 'sleep_feature_data.csv', index=False)

In [28]:
all_band_info

,id,quartile,stage,chan,band,power
0,1,Q1,n1,C3-M2,SWA,114.670967
1,1,Q1,n1,C3-M2,alpha,6.867511
2,1,Q1,n1,C3-M2,beta,0.742699
3,1,Q1,n1,C3-M2,delta,63.947330
4,1,Q1,n1,C3-M2,fastsigma,1.540534
5,1,Q1,n1,C3-M2,sigma,2.486958
6,1,Q1,n1,C3-M2,slowsigma,3.849021
7,1,Q1,n1,C3-M2,theta,23.075504
8,1,Q1,n1,C4-M1,SWA,92.102799
9,1,Q1,n1,C4-M1,alpha,6.583092


In [ ]:
all_feature_info